In [0]:
#from pyspark.sql.functions import unbase64
from pyspark.sql.types import *
from pyspark.sql.functions import *
# Source with default settings
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# Prepare training data from a list of (label, features) tuples.
training = spark.createDataFrame([
    (1.0, Vectors.dense([50.0, 20.0, 20.0])),
    (0.0, Vectors.dense([60.0, 30.0, 30.0])),
    (0.0, Vectors.dense([70, 25.0, 250.0])),
    (1.0, Vectors.dense([65, 18.0, 18.0]))], ["label", "features"])

# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Print out the parameters, documentation, and any default values.
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

# Since model1 is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())



#connectionString = "Endpoint=sb://iothub-ns-ks-bigdata-5809426-7048d14abf.servicebus.windows.net/;SharedAccessKeyName=iothubowner;SharedAccessKey=sQQPHOQhXdvNxTBcBjTfz2fWOTTTnRwRZu+/9HKbXmM=;EntityPath=ks-bigdata-iot-hub-test"
connectionString = "Endpoint=sb://iothub-ns-ks-bigdata-5809426-7048d14abf.servicebus.windows.net/;SharedAccessKeyName=iothubowner;SharedAccessKey=sQQPHOQhXdvNxTBcBjTfz2fWOTTTnRwRZu+/9HKbXmM=;EntityPath=ks-bigdata-iot-hub-test"


ehConf = {
  'eventhubs.connectionString' : connectionString
}

ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)

# Schema of incoming data from IoT hub
schema = "timestamp timestamp, temperature double, humidity double, deviceId string, status string"

# Read directly from IoT Hub using the EventHubs library for Databricks
iot_stream = (
  spark.readStream.format("eventhubs")                                              # Read from IoT Hubs directly
    .options(**ehConf)                                                              # Use the Event-Hub-enabled connect 
    .load()                                                                         # Load the data
    .withColumn('reading', F.from_json(F.col('body').cast('string'), schema))       # Extract the "body" payload from  
)

#display(iot_stream)

iot_stream = iot_stream.withColumn('temperature', iot_stream.reading.temperature).withColumn('humidity', iot_stream.reading.humidity).withColumn('deviceId', iot_stream.reading.deviceId).withColumn('ts', iot_stream.reading.timestamp)
iot_stream2 = iot_stream.groupBy(window(iot_stream.ts, "5 seconds", "3 seconds"), iot_stream.deviceId).agg(avg('humidity'), avg('temperature'), min('humidity'), min('temperature'), max('humidity'), max('temperature'))

assembler = VectorAssembler(
    inputCols=["avg(humidity)", 'avg(temperature)', 'avg(temperature)'],
    outputCol="features")

features = assembler.transform(iot_stream2.withColumn('tsp', iot_stream2.window.start))
#print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
#output.select("features", "clicked").show(truncate=False)

predictions = model1.transform(features)
predictions.printSchema
display(predictions)



window,deviceId,avg(humidity),avg(temperature),min(humidity),min(temperature),max(humidity),max(temperature),tsp,features,rawPrediction,probability,prediction
"List(2020-11-10T17:50:36.000+0000, 2020-11-10T17:50:41.000+0000)",iot_sensor_test_0,71.7434151640796,27.27711782647488,65.33488119093086,22.111996709190397,78.12755484654772,33.82690353808684,2020-11-10T17:50:36.000+0000,"List(1, 3, List(), List(71.7434151640796, 27.27711782647488, 27.27711782647488))","List(1, 2, List(), List(-0.6483949339839299, 0.6483949339839299))","List(1, 2, List(), List(0.343351326382592, 0.6566486736174081))",1.0
"List(2020-11-10T17:59:21.000+0000, 2020-11-10T17:59:26.000+0000)",iot_sensor_test_5,79.2575694147803,23.726011198468072,79.12812640601345,21.9843021418479,79.38701242354716,25.467720255088246,2020-11-10T17:59:21.000+0000,"List(1, 3, List(), List(79.2575694147803, 23.726011198468072, 23.726011198468072))","List(1, 2, List(), List(-3.6049072705871836, 3.6049072705871836))","List(1, 2, List(), List(0.026470241051388385, 0.9735297589486117))",1.0
"List(2020-11-10T17:58:45.000+0000, 2020-11-10T17:58:50.000+0000)",iot_sensor_test_3,66.77903176369071,25.57119584399593,62.86907046054363,21.29214637847772,72.61849098116478,29.487521913417417,2020-11-10T17:58:45.000+0000,"List(1, 3, List(), List(66.77903176369071, 25.57119584399593, 25.57119584399593))","List(1, 2, List(), List(-1.228107820728825, 1.228107820728825))","List(1, 2, List(), List(0.22651277293472696, 0.7734872270652731))",1.0
"List(2020-11-10T17:42:42.000+0000, 2020-11-10T17:42:47.000+0000)",iot_sensor_test_1,65.5343919730732,34.982444548558874,65.5343919730732,34.982444548558874,65.5343919730732,34.982444548558874,2020-11-10T17:42:42.000+0000,"List(1, 3, List(), List(65.5343919730732, 34.982444548558874, 34.982444548558874))","List(1, 2, List(), List(4.777040126786947, -4.777040126786947))","List(1, 2, List(), List(0.9916494323115539, 0.008350567688446092))",0.0
"List(2020-11-10T17:47:39.000+0000, 2020-11-10T17:47:44.000+0000)",iot_sensor_test_9,76.76237896996649,29.196835188049572,72.75320714455233,24.5651538006368,78.91860732263223,34.621898293993326,2020-11-10T17:47:39.000+0000,"List(1, 3, List(), List(76.76237896996649, 29.196835188049572, 29.196835188049572))","List(1, 2, List(), List(0.05961417788499723, -0.05961417788499723))","List(1, 2, List(), List(0.5148991322922173, 0.48510086770778277))",0.0
"List(2020-11-10T17:49:42.000+0000, 2020-11-10T17:49:47.000+0000)",iot_sensor_test_2,72.08596992908097,29.531989288417726,60.1398662013694,25.07120005140061,79.01548197990778,33.198255863702244,2020-11-10T17:49:42.000+0000,"List(1, 3, List(), List(72.08596992908097, 29.531989288417726, 29.531989288417726))","List(1, 2, List(), List(0.7275802167311749, -0.7275802167311749))","List(1, 2, List(), List(0.6742740433542144, 0.32572595664578563))",0.0
"List(2020-11-10T17:53:06.000+0000, 2020-11-10T17:53:11.000+0000)",iot_sensor_test_1,68.14626185483641,28.48606287135347,61.3066745177816,21.07918050606671,78.65155754782243,34.382643840795154,2020-11-10T17:53:06.000+0000,"List(1, 3, List(), List(68.14626185483641, 28.48606287135347, 28.48606287135347))","List(1, 2, List(), List(0.45998668418960875, -0.45998668418960875))","List(1, 2, List(), List(0.6130110172542309, 0.3869889827457692))",0.0
"List(2020-11-10T17:44:21.000+0000, 2020-11-10T17:44:26.000+0000)",iot_sensor_test_7,67.06628861530139,26.26052365372205,62.894755861012236,22.17788004365764,69.19747437936904,34.04721768311675,2020-11-10T17:44:21.000+0000,"List(1, 3, List(), List(67.06628861530139, 26.26052365372205, 26.26052365372205))","List(1, 2, List(), List(-0.8253589627981928, 0.8253589627981928))","List(1, 2, List(), List(0.3046272864090924, 0.6953727135909076))",1.0
"List(2020-11-10T17:50:33.000+0000, 2020-11-10T17:50:38.000+0000)",iot_sensor_test_5,64.74966240082645,29.307532867183816,64.54934198434613,27.660754292802288,64.94998281730676,30.954311441565345,2020-11-10T17:50:33.000+0000,"List(1, 3, List(), List(64.7496624008264